# Nifti Read Example

The purpose of this notebook is to illustrate reading Nifti files and iterating over patches of the volumes loaded from them.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import torch
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import glob
import tempfile

import nibabel as nib

sys.path.append('..')

from monai import application, data, networks, utils

application.config.print_config()

MONAI version: 0.0.1
Python version: 3.7.3 (default, Mar 27 2019, 22:11:17)  [GCC 7.3.0]
Numpy version: 1.16.4
Pytorch version: 1.3.1
Ignite version: 0.2.1


Define a function for creating test images and segmentations:

In [2]:
def create_test_image_3d(height, width, depth, numObjs=12, radMax=30, noiseMax=0.0, numSegClasses=5):
    '''Return a noisy 3D image and segmentation.'''
    image = np.zeros((width, height,depth))

    for i in range(numObjs):
        x = np.random.randint(radMax, width - radMax)
        y = np.random.randint(radMax, height - radMax)
        z = np.random.randint(radMax, depth - radMax)
        rad = np.random.randint(5, radMax)
        spy, spx, spz = np.ogrid[-x:width - x, -y:height - y, -z:depth - z]
        circle = (spx * spx + spy * spy + spz * spz) <= rad * rad

        if numSegClasses > 1:
            image[circle] = np.ceil(np.random.random() * numSegClasses)
        else:
            image[circle] = np.random.random() * 0.5 + 0.5

    labels = np.ceil(image).astype(np.int32)

    norm = np.random.uniform(0, numSegClasses * noiseMax, size=image.shape)
    noisyimage = utils.arrayutils.rescale_array(np.maximum(image, norm))

    return noisyimage, labels

Create a number of test Nifti files 

In [3]:
tempdir=tempfile.mkdtemp()

for i in range(5):
    im,seg=create_test_image_3d(256,256,256)
    n=nib.Nifti1Image(im,np.eye(4))
    nib.save(n,os.path.join(tempdir,'im%i.nii.gz'%i))
    n=nib.Nifti1Image(seg,np.eye(4))
    nib.save(n,os.path.join(tempdir,'seg%i.nii.gz'%i))

Create a stream generator which yields the file paths according to a defined glob pattern:

In [4]:
names=os.path.join(tempdir,'im*.nii.gz')
gsrc=data.streams.GlobPathGenerator(names,do_once=True)

print(list(gsrc))

[('/tmp/tmpm9s_qy4p/im0.nii.gz',), ('/tmp/tmpm9s_qy4p/im1.nii.gz',), ('/tmp/tmpm9s_qy4p/im2.nii.gz',), ('/tmp/tmpm9s_qy4p/im3.nii.gz',), ('/tmp/tmpm9s_qy4p/im4.nii.gz',)]


Create a reader which loads the nifti file names as they come from the source:

In [5]:
src=data.readers.NiftiCacheReader(gsrc,5,image_only=False)

for im in src:
    vol,header=im
    print(vol.shape,vol.dtype, header['filename_or_obj'])

(256, 256, 256) float32 /tmp/tmpm9s_qy4p/im0.nii.gz
(256, 256, 256) float32 /tmp/tmpm9s_qy4p/im1.nii.gz
(256, 256, 256) float32 /tmp/tmpm9s_qy4p/im2.nii.gz
(256, 256, 256) float32 /tmp/tmpm9s_qy4p/im3.nii.gz
(256, 256, 256) float32 /tmp/tmpm9s_qy4p/im4.nii.gz


Alternatively create a path generator which reads two sets of names for the images and segmentation, then the reader to load these pairs:

In [6]:
names=os.path.join(tempdir,'im*.nii.gz')
segs=os.path.join(tempdir,'seg*.nii.gz')

gsrc=data.streams.GlobPathGenerator(names,segs,do_once=True)

src=data.readers.NiftiCacheReader(gsrc,5,image_only=True)

for im,seg in src:
    print(im.shape,seg.shape)

(256, 256, 256) (256, 256, 256)
(256, 256, 256) (256, 256, 256)
(256, 256, 256) (256, 256, 256)
(256, 256, 256) (256, 256, 256)
(256, 256, 256) (256, 256, 256)


Filenames don't need to come from generators, a list of names also works:

In [7]:
images=glob.glob(os.path.join(tempdir,'im*.nii.gz'))
print(images)

src=data.readers.NiftiCacheReader(images,5,image_only=True)

print('Number of loaded images:',len(tuple(src)))

['/tmp/tmpm9s_qy4p/im4.nii.gz', '/tmp/tmpm9s_qy4p/im2.nii.gz', '/tmp/tmpm9s_qy4p/im1.nii.gz', '/tmp/tmpm9s_qy4p/im0.nii.gz', '/tmp/tmpm9s_qy4p/im3.nii.gz']
Number of loaded images: 5


The stream transforms can then be applied to the images coming from the Nifti sources, eg. selecing each 2D image in the XY dimension:

In [8]:
dimsrc=data.transforms.patch_streams.select_over_dimension(src)

for xy in dimsrc:
    print(xy[0].shape,xy[1].shape)
    break # only need to see one

(256,) (256,)


We can also sample uniform patches from the read volumes:

In [9]:
randsrc=data.transforms.patch_streams.uniform_random_patches(src)

for patches in randsrc:
    print(patches[0].shape,patches[1].shape)
    break # only need to see one

(64, 64) (64, 64)


Putting it all together into a stream which loads the images only, iterates through the depth dimension of each, and selects 2 random patches from each 2D image:

In [10]:
gsrc=data.streams.GlobPathGenerator(names,segs,do_once=True)
src=data.readers.NiftiCacheReader(gsrc,5,image_only=True)
src=data.transforms.patch_streams.select_over_dimension(src)
src=data.transforms.patch_streams.uniform_random_patches(src,(25,44),2)

for im in src:
    print(im[0].shape)
    break
    
# expected size is 5 * 256 * 2:
print('Number of 2D images:',len(tuple(src)))

(25, 44)
Number of 2D images: 2560
